# Set up

Load libraries

In [1]:
import os
import sys, getopt
from pathlib import Path
import xml.etree.cElementTree as ET
import pandas as pd

Libraries for database access

In [2]:
from configparser import ConfigParser
import psycopg2
from psycopg2.extras import DictCursor
from psycopg2.extensions import AsIs

Here some instructions [on how to connect to gsheets API](https://medium.com/analytics-vidhya/how-to-read-and-write-data-to-google-spreadsheet-using-python-ebf54d51a72c).

In [3]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow,Flow
from google.auth.transport.requests import Request
import os

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']


# Read data from different sources

## Read invetory from Google Spreadsheet

In [6]:
def read_gsheet(SPREADSHEET_ID,RANGE_NAME, JSONFILE):
    global values_input, service
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(JSONFILE, SCOPES) 
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result_input = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                                range=RANGE_NAME).execute()
    values_input = result_input.get('values', [])

    if not values_input and not values_expansion:
        print('No data found.')

In [27]:
# 
conf_json = Path(os.path.expanduser('~')) / '.secrets/googlecloud/python-gsheet-api.json'
Lilas_doc='1zwNuc82Opu3Kxd93YRzbzUXjP3RZzJyR' # this does not work, maybe due to format (XLSX instead of gsheets?)
my_doc='105V_zTZ-Hzj1ZuZDt1HFo750viegjmM2zyfPvJmPDF4'
read_gsheet(my_doc,'Assessment Target!A1:AA10000',conf_json)

In [28]:
values_input[10]

['Strategic',
 'English_SWA_2015',
 'English_SWA_2015_1',
 'Shrublands on southern Swan Coastal Plain Ironstone',
 '2015',
 'English',
 'IUCN RLE 2.0',
 'Sub-global/Sub-national',
 'Terrestrial',
 'T3. Shrublands and shrubby woodlands',
 'T3.2 Seasonally dry temperate heath and shrublands',
 'DB Public',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 'CR',
 '',
 'Oceania',
 'AU',
 '',
 '',
 'too expensive']

In [49]:

inventory=pd.DataFrame(values_input[1:], columns=values_input[0])

In [50]:
# inventory.columns
inventory[inventory['Ecosystem ID']=='Benson_VineThicket_2013_1'][['Reference label', 'Ecosystem ID', 'AT-Name',
       'Realm\nGET_Level 1', 'Biome\nGET_Level 2',
       'Functional Group\nGET_Level 3']]

,Reference label,Ecosystem ID,AT-Name,Realm\nGET_Level 1,Biome\nGET_Level 2,Functional Group\nGET_Level 3
2,Keith_Foundations_2013,Benson_VineThicket_2013_1,Mock Olive - Wilga - Peach Bush - Carissa dry ...,Terrestrial,T1. Tropical-subtropical forests,T1.2 Tropical-subtropical dry forests and scrubs


In [86]:
selected_columns = ['Reference label', 'Ecosystem ID', 'AT-Name',
       'Realm\nGET_Level 1', 'Biome\nGET_Level 2',
       'Functional Group\nGET_Level 3']
inv_df = inventory[selected_columns]

## Read data from GET database

In [51]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'iucnecoadm'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

In [52]:
conn = psycopg2.connect(**db)
cur = conn.cursor(cursor_factory=DictCursor)

In [53]:
qry = """
SELECT eco_id,eco_name,eco_name_orig,external_code,efg_code,level,membership,assigned_by 
FROM rle.assessment_get_xwalk
LEFT JOIN rle.assessment_units USING(eco_id)
"""
cur.execute(qry)
xwalks = cur.fetchall()
xwalk_df=pd.DataFrame(xwalks,
                columns=['eco_id','eco_name','eco_name_orig','external_code','efg_code','level','membership','assigned_by'])

Close database connection


In [54]:
cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.


## Read data from collection of XML files

Define function to read selected nodes from XML

In [55]:
def extract_IUCN_GET_classification(file):
    records=list()
    tree = ET.parse(file)
    root = tree.getroot()
    for CS in root:
        record={'id':CS.attrib['id'],'name':CS.attrib['name']}
        for AT in CS.findall('Assessment-Target'):
            record['AT-id']=AT.find('AT-id').text

            for elem in AT.findall("Classifications/Classification-system[@id='IUCN Global Typology']"):
                record['version']=elem.attrib['version']
                record['assigned-by']=elem.attrib['assigned-by']
                for subelem in elem:
                    record['level' + subelem.attrib['level']]=subelem.text
            records.append(record)
    return(records)

Path to the XML collection

In [56]:
xmldb = Path(os.path.expanduser('~')) / 'proyectos' / 'IUCN-RLE' / 'RLE_DB' 

Iterate through folders to read all xml files:

In [57]:
records=list()
for pub_status in ['3.Published','2.Not published' ]:
    for asm_type in ['Strategic','Systematic']:
        xmldir = xmldb / pub_status / asm_type
        for xmlfile in os.listdir(xmldir):
            new_records=extract_IUCN_GET_classification(xmldir / xmlfile)
            records.extend(new_records)
            

In [58]:
print(xmlfile)
len(records)

Pliscoff_RLE_Chile_2015.xml


638

In [59]:
records[10]

{'id': 'Oliveira_TepuiShrublands_2013_1',
 'name': 'Tepui shrublands, Southern Venezuela, LC',
 'AT-id': 'VEN_Tepui_Shrublands_1',
 'version': 'draft',
 'assigned-by': 'RLE team',
 'level1': 'Terrestrial',
 'level2': 'T3. Shrublands and shrubby woodlands',
 'level3': 'T3.1 Seasonally dry tropical shrublands'}

In [66]:
xml_df=pd.DataFrame(records)


# Compare records from different sources

We can compare records from the inventory and the information extracted from the xml files, they should match like this example, where both sources use the same id.

In [71]:
inv_df[inv_df['Ecosystem ID']=='Benson_VineThicket_2013_1']

,Reference label,Ecosystem ID,AT-Name,Realm\nGET_Level 1,Biome\nGET_Level 2,Functional Group\nGET_Level 3
2,Keith_Foundations_2013,Benson_VineThicket_2013_1,Mock Olive - Wilga - Peach Bush - Carissa dry ...,Terrestrial,T1. Tropical-subtropical forests,T1.2 Tropical-subtropical dry forests and scrubs


In [130]:
xml_df[xml_df['id']=='Benson_VineThicket_2013_1']
xml_df[xml_df['id']=='Herrera_RLE_CostaRica_2015_24']


,id,name,AT-id,version,assigned-by,level1,level2,level3,inventory
253,Herrera_RLE_CostaRica_2015_24,10.4. Páramos o vegetación paramera de la Cord...,CRI_10.4_24,Draft,RLE team,Terrestrial,T3. Shrublands and shrubby woodlands,NaN,True


We expect to have many matches between the inventory (inv_df) and the table from the xml collection (xml_df):

In [104]:
xml_df['inventory']=xml_df['id'].isin(inv_df['Ecosystem ID'].unique())
print("There are {} matches out of {} rows".format(xml_df['inventory'].sum(),xml_df.shape[0]))
qry=xml_df[xml_df['inventory']]['level3'].isnull().sum()
print("But {} of the matches do not have the value for level 3".format(qry))

There are 484 matches out of 638 rows
But 143 of the matches do not have the value for level 3


In [106]:
inventory.columns

Index(['CS Type', 'Reference label', 'Ecosystem ID', 'AT-Name', 'Year',
       'Language', 'Assessment Protocol', 'Scope of Assessment',
       'Realm\nGET_Level 1', 'Biome\nGET_Level 2',
       'Functional Group\nGET_Level 3', 'Status', 'CO', 'CR', 'EN', 'VU', 'NT',
       'LC', 'DD', 'NE', 'Overall_RC', 'Outcome_RA', 'Region', 'Countries',
       'Asessment area (km2)', 'Requiere revisión ', 'Publisher's permission',
       'xml table'],
      dtype='object')

We can also do the match the other way around:

In [114]:
inventory['xml table']=inventory['Ecosystem ID'].isin(xml_df['id'].unique())
print("There are {} matches out of {} rows".format(inventory['xml table'].sum(),inv_df.shape[0]))
qry=(inventory[inventory['xml table']]['Functional Group\nGET_Level 3']=="").sum()
if qry>0:
    print("But {} of the matches do not have the value for level 3".format(qry))
else:
    print("All matches have a value for level 3!")

There are 484 matches out of 5290 rows
But 235 of the matches do not have the value for level 3


In [115]:
(inventory['Functional Group\nGET_Level 3']=="").sum()

5015

In [116]:
inventory[inventory['xml table']]['Functional Group\nGET_Level 3']

0           T1.3 Tropical-subtropical montane rainforests
1       T3.2 Seasonally dry temperate heath and shrubl...
2        T1.2 Tropical-subtropical dry forests and scrubs
3                                    M1.9 Upwelling zones
4                                 M1.3 Photic coral reefs
                              ...                        
1801              TM2.1 Coastal shrublands and grasslands
1802              TM2.1 Coastal shrublands and grasslands
1803              TM2.1 Coastal shrublands and grasslands
1804              TM2.1 Coastal shrublands and grasslands
1805              TM2.1 Coastal shrublands and grasslands
Name: Functional Group\nGET_Level 3, Length: 484, dtype: object

In [126]:
def changeid(x):
    return x.replace(':','_')

xwalk_df['new_id'] = xwalk_df['eco_id'].apply(changeid)

xwalk_df['inventory']=xwalk_df['new_id'].isin(inventory['Ecosystem ID'].unique())
print("There are {} matches out of {} rows".format(xwalk_df['inventory'].sum(),xwalk_df.shape[0]))


There are 137 matches out of 545 rows


In [129]:
xwalk_df[xwalk_df['inventory']]

,eco_id,eco_name,eco_name_orig,external_code,efg_code,level,membership,assigned_by,inventory,new_id
1,Herrera_RLE_CostaRica_2015:24,10.4. Páramos o vegetación paramera de la cord...,10.4. Páramos o vegetación paramera de la cord...,Herrera_RLE_CostaRica_2015:24,T6.5,6.0,0.10,JR Ferrer-Paris,True,Herrera_RLE_CostaRica_2015_24
2,Herrera_RLE_CostaRica_2015:29,11.5 Páramos de la Cordillera de Talamanca,11.5 Páramos de la Cordillera de Talamanca,Herrera_RLE_CostaRica_2015:29,T6.5,6.0,0.10,JR Ferrer-Paris,True,Herrera_RLE_CostaRica_2015_29
4,Holdaway_Uncommon_NewZealand_2012:58,Snowbanks,Snowbanks,Holdaway_Uncommon_NewZealand_2012:58,T6.1,6.0,0.01,JR Ferrer-Paris,True,Holdaway_Uncommon_NewZealand_2012_58
5,FerrerParis_Continental_ForestMacrogroup_2019:1,1.A.1.Ea.134 - Caribbean Coastal Lowland Dry F...,1.A.1.Ea.134 - Caribbean Coastal Lowland Dry F...,FerrerParis_Continental_ForestMacrogroup_2019:1,T1.2,5.0,0.10,JR Ferrer Paris,True,FerrerParis_Continental_ForestMacrogroup_2019_1
6,FerrerParis_Continental_ForestMacrogroup_2019:2,1.A.1.Ea.294 - Caribbean Dry Limestone Forest,1.A.1.Ea.294 - Caribbean Dry Limestone Forest,FerrerParis_Continental_ForestMacrogroup_2019:2,T1.2,5.0,0.10,JR Ferrer Paris,True,FerrerParis_Continental_ForestMacrogroup_2019_2
...,...,...,...,...,...,...,...,...,...,...
267,Holdaway_Uncommon_NewZealand_2012:19,Cushion bogs,Cushion bogs,Holdaway_Uncommon_NewZealand_2012:19,TF1.6,6.0,0.10,JRFP,True,Holdaway_Uncommon_NewZealand_2012_19
268,Holdaway_Uncommon_NewZealand_2012:47,Recent lava flows,Recent lava flows,Holdaway_Uncommon_NewZealand_2012:47,T3.4,6.0,0.10,JRFP,True,Holdaway_Uncommon_NewZealand_2012_47
269,Holdaway_Uncommon_NewZealand_2012:26,Estuary,Estuary,Holdaway_Uncommon_NewZealand_2012:26,FM1.2,6.0,0.10,JRFP,True,Holdaway_Uncommon_NewZealand_2012_26
299,Etter_RLE_Colombia_2015_44,None,Ha_ZBH-B35 Halobiomas del Zonobioma de Bosque ...,Etter_RLE_Colombia_2015_44,MFT1.2,6.0,1.00,Provita,True,Etter_RLE_Colombia_2015_44


In [74]:
for i in xml_df['id'].unique():
    if i in inv_df['Ecosystem ID'].unique():
        xml_df[xml_df['id']]['inventory']
    else:
        print('missing')

Kingsford_MurrayDarlingBasin_2013_1
Keith_CaribbeanCoralReefs_2013_1
Bland_MesoAmericaReef_2017_1
Williams_AlpineSnowPatch_2015_1
Riecken_RaisedBogs_2013_1
missing
Clark_IvertebrateComm_2015_1
Wardle_GeorginaGidgee_2015_1
English_SWA_2015_1
Bland_MarineEcosystem_2018_1
missing
missing
Essl_GermanTamarisk_2013_1
Keith_CoastalSandstone_2013_1
Murray_TidalFlats_2015_1
Tozer_CumberlandPlain_2015_1
Sievers_Seagrasses_MoretonBay_2020_1
Keith_GonakierForest_2013_1
Metcalfe_WetTropics_2015_1
missing
Moat_TapiaForest_2013_1
Marshall_FringeMangroves_2018_1
Burns_MountainAshForest_2015_1
Sievers_Saltmarshes_MoretonBay_2020_1
Keith_GiantKelp_2013_1
Keith_CapeFlatsSandFynbos_2013_1
Sievers_SundarbansMangrove_2020_1
Bonifacio_KRSWetland_2013_1
Barrett_MountainSummit_2015_1
missing
Pisanu_LakeEyre_2015_1
Ghoraba_LakeBurullus_2019_1
Sievers_Mangroves_MoretonBay_2020_1
Bonifacio_SeagrassCommunity_2013_1
Keith_CoolibahBlackBox_2013_1
Lester_CoorongLagoons_2013_1
Ghoraba_BurullusSandPlain_2021_1
Mahoney_

In [48]:
df_from_xml['assigned-by'].unique()
df_from_xml['version'].unique()

array(['draft', 'Draft', nan], dtype=object)

In [16]:
list1=df_from_sql['eco_id'].unique()
list2=df_from_xml['id'].unique()

In [19]:
for j in list1:
    if j in list2:
        print(j)

Etter_RLE_Colombia_2015_44
Etter_RLE_Colombia_2015_72


In [34]:
df_from_sql[df_from_sql['eco_id']=='Ghoraba_Egypt_MarineBar_2021:1']

,eco_id,eco_name,eco_name_orig,external_code,efg_code,level,membership,assigned_by
290,Ghoraba_Egypt_MarineBar_2021:1,None,Sand plain ecosystem,None,MT2.1,6.0,1.00,JRFEP


In [35]:
df_from_xml[df_from_xml['id']=='Ghoraba_Egypt_MarineBar_2021_1']

,id,name,AT-id,version,assigned-by,level1,level2,level3


In [33]:
df_from_sql[df_from_sql['eco_id']!=df_from_sql['external_code']]

,eco_id,eco_name,eco_name_orig,external_code,efg_code,level,membership,assigned_by
288,IUCN_FR_Mangroves_Mayotte_2017:2,None,Mangroves mésohalines et mangroves estuariennes,None,MFT1.2,6.0,1.00,JRFEP
289,IUCN_FR_Mangroves_Mayotte_2017:3,None,"Tannes, prés-salés, mangroves et forêts supral...",None,MFT1.2,6.0,1.00,JRFEP
290,Ghoraba_Egypt_MarineBar_2021:1,None,Sand plain ecosystem,None,MT2.1,6.0,1.00,JRFEP
291,Ghoraba_Egypt_MarineBar_2021:2,None,Salt marshes,None,MFT1.3,6.0,1.00,JRFEP
292,Uribe_Marine_Colombia_2020:1,None,Manglares Caribe oceánico,None,MFT1.2,6.0,1.00,JRFEP
293,Uribe_Marine_Colombia_2020:5,None,Manglares Pacífico continental norte,None,MFT1.2,6.0,1.00,JRFEP
294,Uribe_Marine_Colombia_2020:9,None,Manglares Pacífico continental sur,None,MFT1.2,6.0,1.00,JRFEP
295,Uribe_Marine_Colombia_2020:10,None,Manglares Caribe continental,None,MFT1.2,6.0,1.00,JRFEP
296,CaseStudies_RLE_Venezuela_2010:1,None,Laguna de Tacarigua,None,MFT1.2,6.0,1.00,JRFEP
297,CaseStudies_RLE_Venezuela_2010:2,None,Ciénaga de Los Olivitos,None,MFT1.2,6.0,1.00,JRFEP
